In [1]:
# python notebook for make your own neural network
# code for 3 layer neural network, code for learning MNIST dataset

In [2]:
# library for numerical arrays
import numpy
# library for data manipulation and handling
import pandas 
# scipy.special for the sigmoid function expit()
import scipy.special
# library for plotting arrays
import matplotlib.pyplot
# ensure the plots are inside this notebook, not an external window
%matplotlib inline

In [3]:
# Neural Network class definition
class NeuralNetwork:
    
    
    # initialise the neural network
    def __init__(self, input_nodes, hidden_nodes, ouput_nodes, learning_rate):
        # set  number of nodes in each input, hidden, output layer
        self.inodes = input_nodes
        self.hnodes = hidden_nodes
        self.onodes = output_nodes
        
        # weight inside the arrays are w_i_j, where link is from node i to node j in the next layer
        # w11, w12, w21, w22, etc
        self.wih = numpy.random.normal(0.0, pow(self.inodes, -0.5), (self.hnodes, self.inodes))
        self.who = numpy.random.normal(0.0, pow(self.hnodes, -0.5), (self.onodes, self.hnodes)) 
        
        # activation function is sigmoid function 
        self.activation_function = lambda x : scipy.special.expit(x)
        
        # learning rate
        self.lr = learning_rate
    
    # train the neural network
    def train(self, inputs_list, targets_list):
        
        # convert input list into numpy 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T
        
        # convert target list into numpy 2d array
        targets = numpy.array(targets_list, ndmin=2).T
        
        # calculate the signal into hidden array
        hidden_inputs = numpy.dot(self.wih, inputs)
        
        # calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs) 
        
        # calculate the signals into final output layer
        final_inputs = numpy.dot(self.who,  hidden_outputs)
        
        # calculate the signals emerging from output layer
        final_outputs = self.activation_function(final_inputs)
        
        # output layer error is the target - actual
        output_errors =  targets - final_outputs 
        
        # hidden layer error is the output errors split by weights, recombined at the hidden nodes
        hidden_errors =  numpy.dot(self.who.T, output_errors) 
        
        # update the weights for the links between the hidden and output layers
        self.who += self.lr * numpy.dot((output_errors * final_outputs * (1.0 - final_outputs)), numpy.transpose(hidden_outputs))
        
        # update the weights for the links between the input and hidden layers
        self.wih += self.lr * numpy.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), numpy.transpose(inputs))
        
        
    
    # query the nueral network
    def query(self, inputs_list):
        
        # convert input list into numpy 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T
        
        # calculate the signal into hidden array
        hidden_inputs = numpy.dot(self.wih, inputs)
        
        # calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs) 
        
        # calculate the signals into final output layer
        final_inputs = numpy.dot(self.who,  hidden_outputs)
        
        # calculate the signals emerging from output layer
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs

In [4]:
# number of input, hidden and output nodes
input_nodes = 784
hidden_nodes = 200
output_nodes = 10

# learning rate is 0.3
learning_rate = 0.1

# create a instance of NeuralNetwork 
n = NeuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)

In [5]:
# load the mnist training data CSV file into a list
training_data_file = open('mnist_train.csv', 'r')
training_data_list = training_data_file.readlines()
training_data_file.close()

In [ ]:
# train the neural network
# epochs is the number of times the training data set is used for training
epochs = 5

for e in range(epochs):
    
    # go through all records in the training data set
    for record in training_data_list:
        # split the record by the ',' commas
        all_values = record.split(',')
        
        # scale and shift the inputs (Feature Scaling)
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        
        # create the target output values (all 0.01, except the desired label which is 0.99) (Feature Scaling)
        targets = numpy.zeros(output_nodes) + 0.01
        
        # all_values[0] is the target label for this record
        targets[int(all_values[0])] = 0.99
        
        # training the network with inputs and target
        n.train(inputs, targets)

In [191]:
# load the mnist test data CSV file into a list
test_data_file = open('mnist_test.csv', 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()

In [192]:
# test for the nueral network
# scorecard for  how well the network performs, intially empty
scorecard = []

for records in test_data_list:
    # split the record by the ',' commas
    all_values = records.split(',') 
    
    # correct answer is the first value
    correct_label = int(all_values[0])
       
    # scale and shift the value
    inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
    outputs = n.query(inputs)
    
    # the index of the highest value corresponds to the label
    label = numpy.argmax(outputs) 
        
    if(label == correct_label):
        # network answer matches with  correct answer, add 1 to scorecard
        scorecard.append(1)
    else:
        # network answer doesn't match with correct answer, add 0 to scorecard
        scorecard.append(0)
    

In [193]:
#  calculate the performance score, the fraction of the scorecard
scorecard_array = numpy.asarray(scorecard)
print("performance", scorecard_array.sum() / scorecard_array.size)


performance 0.9733
